In [1]:
# import csv

# orderers = []
# summary = []

# with open('D:\\me\\masters\\data\\google_books_1299.csv', encoding='utf-8') as file:
#     data = csv.reader(file)
#     for k in data:
#         print(k)

In [25]:
import csv 
import json
from bson.objectid import ObjectId

def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
    unusedColumns = ['','rating','voters','published_date','page_count', 'generes', 'currency']
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf)   

        #convert each csv row into python dict
        for row in csvReader: 
            row['price'] = round(float(row['price']) * 0.27, 2)
            row['genres'] = [element.strip() for element in list(row['generes'].split(','))]
            row['pageCount'] = int(row['page_count'])
            row['publishedDate'] = row['published_date']
            row['imageUrl'] = 'https://mastersimages.blob.core.windows.net/images/book.jpg'

            for column in unusedColumns:
                del row[column]
            
            #add this python dict to json array
            jsonArray.append(row)

    # print(jsonArray)

    result = list({v['title']:v for v in jsonArray}.values())
    print(len(result))
  
    # convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(result, indent=4)
        jsonf.write(jsonString)
          
csvFilePath = r'D:\\me\\university\\masters\\data\\google_books_1299.csv'
jsonFilePath = r'D:\\me\\university\\masters\\data\\books.json'
csv_to_json(csvFilePath, jsonFilePath)

246


In [1]:
from pymongo import MongoClient
from config import *

client = MongoClient()
client = MongoClient(connectionString)
db = client['masters']
collection = db.books
collection

Collection(Database(MongoClient(host=['masters-shard-00-02.cpq5u.mongodb.net:27017', 'masters-shard-00-01.cpq5u.mongodb.net:27017', 'masters-shard-00-00.cpq5u.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-wrhudr-shard-0', tls=True), 'masters'), 'books')

In [1]:
import datetime
str = datetime.datetime.utcnow().strftime('%d.%m.%Y %H:%M:%S')
print(str)

datetime.datetime.strptime(str, '%d.%m.%Y %H:%M:%S')

23.02.2022 16:06:01


'23.02.2022 16:06:01'

In [15]:
datetime.datetime.utcnow() < (datetime.datetime.utcnow() - datetime.timedelta(days = 1))

False

In [13]:
(datetime.datetime.utcnow() - datetime.timedelta(days = 1)).strftime('%Y-%m-%d-%H:%M:%S')

'2022-02-21-18:09:51'

In [7]:
from bson.objectid import ObjectId
a = ObjectId()
a.__str__()

'62154d12e642272bf9189970'

In [5]:
import re

txt = datetime.datetime.utcnow().strftime('%d.%m.%Y %H:%M:%S')
x = re.search('^23.02.2022 ', txt)
x

<re.Match object; span=(0, 11), match='23.02.2022 '>

In [19]:
a = ['Cognitive Psychology &amp', 'Something']
for i in a:
    if i.endswith(' &amp'):
        i = i.replace(' &amp', '')
        print(i)
a

Cognitive Psychology


['Cognitive Psychology &amp', 'Something']

In [18]:
db['books'].drop()
db['authors'].drop()
db['publishers'].drop()
db['genres'].drop()

In [19]:
import json
import datetime
from bson.objectid import ObjectId

def readJsonFile(fileName):
    with open(fileName, encoding='utf-8') as file: 
        # print(json.load(file))
        return json.load(file)

def preprocess_data(data):
    authors = []
    publishers = []
    genres = []

    for record in data:
        date = record['publishedDate']
        record['publishedDate'] = datetime.datetime.strptime(date, '%b %d, %Y')

        record_genres_list = record['genres']

        record_genres_dict_list = []
        for item in record_genres_list:
            if item.endswith(' &amp'):
                item = item.replace(' &amp', '')
            genre_exist = False
            for existing_gerne in genres:
                if existing_gerne['name'] == item:
                    record_genres_dict_list.append(existing_gerne)
                    genre_exist = True
                    break
            if genre_exist == False:
                new_gerne = {'_id': ObjectId(), 'name': item}
                genres.append(new_gerne)
                record_genres_dict_list.append(new_gerne)
        
        record['genres'] = record_genres_dict_list

        author_exist = False
        for author in authors:
            if author['name'] == record['author']:
                author_exist = True
                record['author'] = author
                break
        if author_exist == False:
            new_author = {'_id': ObjectId(), 'name': record['author']}
            authors.append(new_author)
            record['author'] = new_author

        publisher_exist = False
        for publisher in publishers:
            if publisher['name'] == record['publisher']:
                publisher_exist = True
                record['publisher'] = publisher
                break
        if publisher_exist == False:
            new_publisher = {'_id': ObjectId(), 'name': record['publisher']}
            publishers.append(new_publisher)
            record['publisher'] = new_publisher

    return data, authors, publishers, genres

data = readJsonFile('D:\\me\\university\\masters\\data\\books.json')
preprocessed_data = preprocess_data(data)

db.books.insert_many(preprocessed_data[0])
db.authors.insert_many(preprocessed_data[1])
db.publishers.insert_many(preprocessed_data[2])
db.genres.insert_many(preprocessed_data[3])

In [10]:
import json
import datetime
from bson.objectid import ObjectId
import random

books = db.books.find()[0:10]

users = db.users.find()[0:10]

rates = []

for i in range(10):
    dict = {}
    dict['bookId'] = books[i]['_id']
    dict_list = []

    for j in range(10):
        usersRandom = random.randint(0, 9)        

        rate = {}
        rate['user'] = {'_id': users[usersRandom]['_id'], 'name': users[usersRandom]['name']}
        rate['rate'] = random.randint(1, 5)
        rate['comment'] = 'Lorem ipsum dolor sit amet..'
        day_diff = random.randint(0, 3)
        yesterday = datetime.datetime.utcnow() - datetime.timedelta(days = day_diff)
        rate['createdAt'] = datetime.datetime(yesterday.year, yesterday.month, yesterday.day)

        existing = False
        
        # print(rate)
        for item in dict_list:
            # print(item)
            if(item['user']['_id'] == rate['user']['_id']):
                existing = True

        if(existing == False):
            dict_list.append(rate)

    dict['reviews'] = dict_list
    rates.append(dict)

# print(rates)

db['rates'].drop()
db.rates.insert_many(rates)

In [8]:
import json
import datetime
from bson.objectid import ObjectId
import random

books = db.books.find()

users = db.users.find()

rates = []

for book in books:
    dict = {}
    dict['bookId'] = book['_id']
    dict_list = []

    for j in range(20):
        usersRandom = random.randint(0, 119)        

        rate = {}
        rate['user'] = {'_id': users[usersRandom]['_id'], 'name': users[usersRandom]['name']}
        rate['rate'] = random.randint(1, 5)
        rate['comment'] = 'Lorem ipsum dolor sit amet..'
        day_diff = random.randint(0, 3)
        yesterday = datetime.datetime.utcnow() - datetime.timedelta(days = day_diff)
        rate['createdAt'] = datetime.datetime(yesterday.year, yesterday.month, yesterday.day)

        existing = False
        
        # print(rate)
        for item in dict_list:
            # print(item)
            if(item['user']['_id'] == rate['user']['_id']):
                existing = True

        if(existing == False):
            dict_list.append(rate)

    dict['reviews'] = dict_list
    rates.append(dict)

# print(rates)

db['rates'].drop()
db.rates.insert_many(rates)

In [4]:
db['rates'].drop()

In [4]:
any("book11" in item.values() and "book11" in item.values() for item in a)

True

In [7]:
import json
import datetime
from bson.objectid import ObjectId
import random

books = db.books.find()
users = db.users.find()
orders = []

for i in range(300):
    booksAmount = random.randint(1, 5)
    usersRandom = random.randint(0, 119)

    dict = {}
    orderBooks = []
    user = {}
    
    user['_id'] = users[usersRandom]['_id']
    user['name'] = users[usersRandom]['name']

    dict['user'] = user
    day_diff = random.randint(0, 3)
    yesterday = datetime.datetime.utcnow() - datetime.timedelta(days = day_diff)
    dict['createdAt'] = datetime.datetime(yesterday.year, yesterday.month, yesterday.day)

    for i in range(booksAmount):
        booksRandom = random.randint(0, 245)
        amount = random.randint(1, 5)
        book = books[booksRandom]
        new_book = {
            "_id": book['_id'], 
            "title" : book['title'],
            "price" : book['price'],
            "amount":amount 
            }
        orderBooks.append(new_book)

    dict['books'] = orderBooks

    orders.append(dict)
    # print(dict)


print(len(orders))

    
db.orders.insert_many(orders)

300
